In [1]:
from atlassian import Jira
from bidict import bidict

# jira_info = {
#     'url': 'https://tangramcare.atlassian.net/',
#     'username': 'xxx@ownedoutcomes.com',
#     'api_token': 'xxxxxxxxxxxxxxxx',
# }

jira_info = {
    'url': 'https://tangramcare.atlassian.net/',
    'username': 'awaz@ownedoutcomes.com',
    'api_token': '6Pu9J7zSN4wBqwqgSREsAE08',
}



jira = Jira(
            url=jira_info['url'],
            username=jira_info['username'],
            password=jira_info['api_token'],
            cloud=True)

In [41]:
import datetime
def datetime_to_date_converter(date):
    from datetime import datetime
    from dateutil import parser
    if isinstance(date, datetime):
        return date.strftime("%Y-%m-%d")
    return parser.parse(date).strftime("%Y-%m-%d")


def get_schema_from_sheet(sheet): # TODO schema validation
    schema = {}
    schema['offset_row'] = 1 # offset of WBS: TODO search for WBS
    schema['offset_col'] = 0 # offset of WBS: TODO search for WBS
    
    schema['columns'] = { # TODO make it dynamic
        'Category': 'A',
        'WBS': 'C',
        'Name': 'B',
        'Description': 'M',
        'Depends on': 'D',
        'Estimate [h]': 'E',
        'Jira ID': 'V',
        'Start time': 'S',
        'Finish time': 'T',
        # 'Duration [h]': 'L'
    }
    schema['data_offset_row'] = 3

    return schema
    

def wbs_regex_check(string, separator = '.'):
    import re
    return True if re.match('\A([0-9]+\\' + separator + '?)+\Z', string) else False


def wbs_regex_task(string):
    import re
    return True if re.match('\A[0-9]+\Z', string) else False


def check_wbs_level(wbs, separator = '.'):
    return len(wbs.split(separator))


def get_wbs_ancestor(wbs, separator = '.'):
    if len(wbs.split(wbs)) == 1:
        return wbs.split(wbs)[0]
    else:
        return '.'.join(wbs.split('.')[0:-1])


def check_if_successor(wbs, successor):
    wbs_split = wbs.split('.')
    successor_split = successor.split('.')
    if len(wbs_split) >= len(successor_split):
        return False
    for i in range(len(wbs_split)):
        if wbs_split[i] != successor_split[i]:
            return False
    return True


def get_wbs_lowest_level_task_number(wbs, separator = '.'):
    return wbs.split(separator)[-1]


def expand_wbs_range(start, finish, separator = '.'):
    if check_wbs_level(start, separator) != check_wbs_level(finish, separator):
        print('ERROR: WBS range level differs: ', start, ' - ', finish)
        return []
    if get_wbs_ancestor(start, separator) != get_wbs_ancestor(finish, separator):
        print('ERROR: WBS range ancestor differs: ', start, ' - ', finish)
        return []
    lowest_level_task = list(range(int(get_wbs_lowest_level_task_number(start, separator)), 1 + int(get_wbs_lowest_level_task_number(finish, separator))))
    wbs_tasks = []
    for i in lowest_level_task:
        if get_wbs_ancestor(start, separator) == '':
            wbs_tasks.append(str(i))
        else:
            wbs_tasks.append(get_wbs_ancestor(start, separator) + '.' + str(i))
    return wbs_tasks
    

def parse_dependencies(dependency_str, wbs_separator = '.'):
    wbs_tasks = []
    for i in str(dependency_str).split(','):
        if wbs_regex_check(i.strip(), wbs_separator):
            wbs_tasks.append(i)
        else:
            range = i.split('-')
            if len(range) != 2:
                print("ERROR: It is not a range: ", range)
                continue
            if wbs_regex_check(range[0].strip(), wbs_separator) == False or wbs_regex_check(range[1].strip(), wbs_separator) == False:
                print("ERROR: It is not a range: ", range)
                continue
            wbs_tasks = wbs_tasks + expand_wbs_range(range[0].strip(), range[1].strip(), wbs_separator)
    return wbs_tasks


def get_value_for_wbs(schema, sheets, wbs, column_name):
    for sheet in sheets:
        i = schema['data_offset_row']
        while sheet[schema['columns']['WBS'] + str(i)].value is not None:
            if wbs == str(sheet[schema['columns']['WBS'] + str(i)].value):
                return str(sheet[schema['columns'][column_name] + str(i)].value)
            i = i + 1 
    return None


def get_jira_key_for_wbs(schema, sheets, wbs):
    return get_value_for_wbs(schema, sheets, wbs, 'Jira ID')


# def get_start_finish_from_child(schema, wbs):
#     from dateutil import parser
#     min = None
#     max = None
#     i = schema['data_offset_row']
#     while sheet[schema['columns']['WBS'] + str(i)].value is not None:
#         if check_if_successor(wbs, str(sheet[schema['columns']['WBS'] + str(i)].value)):

#             if min is None and sheet[schema['columns']['Start time'] + str(i)].value is not None:
#                 min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)
#             if max is None and sheet[schema['columns']['Finish time'] + str(i)].value is not None:
#                 max = parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value)

#             if sheet[schema['columns']['Start time'] + str(i)].value is not None: 
#                 if parser.parse(sheet[schema['columns']['Start time'] + str(i)].value) < min:
#                     min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)
#             if sheet[schema['columns']['Finish time'] + str(i)].value is not None: 
#                 if parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value) > max:
#                     max = parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value)

#         i = i + 1 
#     return min, max


# def get_min_from_start(schema):
#     from dateutil import parser
#     min = None
#     i = schema['data_offset_row']
#     while sheet[schema['columns']['WBS'] + str(i)].value is not None:
#         if min is None and sheet[schema['columns']['Start time'] + str(i)].value is not None:
#             min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

#         if sheet[schema['columns']['Start time'] + str(i)].value is not None: 
#             if parser.parse(sheet[schema['columns']['Start time'] + str(i)].value) < min:
#                 min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

#         i = i + 1 
#     return min


def create_or_update_issue(jira, sheets, sheet, schema, index, project, type, label, parent = None):

    print("current sheet: ", sheet, "    current row: ", index)

    ### form basic fields for create or update
    task_json = {
        'summary': sheet[schema['columns']['Name'] + str(index)].value,
        'description': sheet[schema['columns']['Description'] + str(index)].value,
        'components': [{"name":"FE"}]
    }

    if sheet[schema['columns']['Start time'] + str(index)].value is not None:
        task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
    if sheet[schema['columns']['Finish time'] + str(index)].value is not None:
        task_json['customfield_11702'] = datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)

    if sheet[schema['columns']['Start time'] + str(index)].value is not None:
        task_json['customfield_11724'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
    if sheet[schema['columns']['Finish time'] + str(index)].value is not None:
        task_json['customfield_11725'] = datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)
    
    if sheet[schema['columns']['Estimate [h]'] + str(index)].value is not None:
        task_json['customfield_10004'] = float(sheet[schema['columns']['Estimate [h]'] + str(index)].value)/4

    ## create issue
    if sheet[schema['columns']['Jira ID'] + str(index)].value is None: 

        task_json['project'] = {'key': project}
        task_json['issuetype'] = {'name': type}
        if parent is not None:
            task_json['parent'] = {'key': parent}

        issue = jira.issue_create(task_json)

        # print(issue)
        sheet[schema['columns']['Jira ID'] + str(index)].value = issue['key']
        sheet[schema['columns']['Jira ID'] + str(index)].hyperlink = jira.url + '/browse/' + issue['key']
        sheet[schema['columns']['Jira ID'] + str(index)].font = Font(underline='single', color='0000FF')

    ## update basic fields     
    else:            
        jira.issue_update(sheet[schema['columns']['Jira ID'] + str(index)].value, task_json)
        pass

    # print(task_json)

    ### section for edit issue

    ## add dependencies
    if sheet[schema['columns']['Depends on'] + str(index)].value is not None:
        if sheet[schema['columns']['Depends on'] + str(index)].value != '':
            # print(parse_dependencies(sheet[schema['columns']['Depends on'] + str(index)].value))
            for dependent_task_wbs in parse_dependencies(sheet[schema['columns']['Depends on'] + str(index)].value):
                dependent_jira_id = get_jira_key_for_wbs(schema, sheets, dependent_task_wbs.strip())
                if dependent_jira_id is not None:
                    # print(dependent_jira_id)
                    fields = {
                        "issuelinks": [
                            {
                                "add": {
                                    "type": {
                                        "name": "Gantt End to Start",
                                        "inward": "has to be done after",
                                        "outward": "has to be done before"
                                    },
                                    "inwardIssue": {
                                        "key": dependent_jira_id
                                    }
                                }
                            }
                        ]
                    }
                    # add dependency
                    jira.edit_issue(issue_id_or_key=sheet[schema['columns']['Jira ID'] + str(index)].value, fields=fields, notify_users=False)

    ## add labels
    labels = [{"add": label}]
    if sheet[schema['columns']['Category'] + str(index)].value is not None:
        labels = labels + [{"add": (str(sheet[schema['columns']['Category'] + str(index)].value)).strip().replace(' ', '_')}]

    jira.edit_issue(issue_id_or_key=sheet[schema['columns']['Jira ID'] + str(index)].value, fields={"labels": labels}, notify_users=False)



def create_or_update_issues(jira, sheets, schema, project, label):
    for sheet in sheets:
        print("current sheet: ", sheet)
        i = schema['data_offset_row']
        none_count = 0
        while none_count < 50:
            if sheet[schema['columns']['WBS'] + str(i)].value is not None:
                wbs = str(sheet[schema['columns']['WBS'] + str(i)].value)
                if not wbs_regex_check(wbs):
                    print('Problem in WBS number structure: ', wbs, ' in sheet: ', sheet)
                    i = i + 1
                    continue

                if sheet[schema['columns']['Name'] + str(i)].value is None or sheet[schema['columns']['Name'] + str(i)].value == '':
                    print('Task do not have any name, wbs: ', wbs, ' in sheet: ', sheet)
                    i = i + 1
                    continue

                # if wbs_regex_task(wbs):
                create_or_update_issue(jira=jira, sheets=sheets, sheet=sheet, schema=schema, index=i, project=project, type='Task', label=label)
                # else:
                #     import re
                #     parent = get_jira_key_for_wbs(schema=schema, wbs=re.match('\A[0-9]+', wbs)[0])
                #     if parent is None:
                #         raise Exception('Task doesn\'t have a parent: ' + wbs)
                #     create_or_update_issue(jira=jira, schema=schema, index=i, project=project, type='Sub-task', label=label, parent=parent)
                # if i > schema['data_offset_row'] + 5: # for tests purposes
                #     return
                none_count = 0
            else:
                none_count = none_count + 1
            i = i + 1
            # if i > 5:
            #     return


In [44]:
from openpyxl import load_workbook
from openpyxl.styles import Font

# filename = 'grouper.xlsx'
# label = 'P2_pipelines'

# filename = 'p2v3.xlsx'
# label = 'P2_v3'

# filename = 'auth.xlsx'
# label = 'Authorization'

# filename = 'tests.xlsx'
# label = 'Tests'

filename = 'fe.xlsx'
label = 'P2v3frontend'


exclude_tab = ["PMSummary"]

workbook = load_workbook(filename=filename, data_only=True)

sheets_to_analyze = []

for sheet in workbook.sheetnames:
    if sheet not in exclude_tab:
        sheets_to_analyze.append(workbook[sheet])
    

print(workbook.sheetnames)
print(sheets_to_analyze)



['PMSummary', 'Components', 'Services', 'Utilities', 'Pipes', 'Directives', 'Misc.', 'Auth0']
[<Worksheet "Components">, <Worksheet "Services">, <Worksheet "Utilities">, <Worksheet "Pipes">, <Worksheet "Directives">, <Worksheet "Misc.">, <Worksheet "Auth0">]


In [45]:
schema = get_schema_from_sheet(sheets_to_analyze)
create_or_update_issues(jira=jira, sheets=sheets_to_analyze, schema=schema, project='PP', label=label)
print("!!! FINISHED !!!")

Updating issue "PP-705" with "{'summary': 'graph-view.component.ts', 'description': 'Several methods are no longer called. This would cut down on the total Unit Test estimation but time would have to be spent on Code Cleanup to determine what should be removed. Move checkGraphFilters method to utils.', 'components': [{'name': 'FE'}], 'customfield_11701': '2021-04-26', 'customfield_11702': '2021-05-07', 'customfield_11724': '2021-04-26', 'customfield_11725': '2021-05-07', 'customfield_10004': 21.625}"
current sheet:  <Worksheet "Components">
current sheet:  <Worksheet "Components">     current row:  3
Updating issue "PP-706" with "{'summary': 'blank-view.component.ts', 'description': None, 'components': [{'name': 'FE'}], 'customfield_11701': '2021-04-19', 'customfield_11702': '2021-04-19', 'customfield_11724': '2021-04-19', 'customfield_11725': '2021-04-19', 'customfield_10004': 0.375}"
current sheet:  <Worksheet "Components">     current row:  4
Updating issue "PP-707" with "{'summary'

In [46]:
workbook.save(filename=filename)
workbook.close()

In [14]:
map = {
    "A": "wbs",
    "B": "name",
    "C": "description",
    "D": "dependency",
    "E": "estimated_development_time_designer",
    "F": "estimated_development_time_reviewer",
    "G": "estimated_development_time_pm",
    "H": "assigned_developer",
    "I": "estimated_testing_time_tester",
    "J": "estimated_testing_time_pm",
    "K": "assigned_tester",
    "L": "actual_development_time_start",
    "M": "actual_development_time_finish",
    "N": "actual_development_time_duration",
    "O": "commit_statistics_insertions",
    "P": "commit_statistics_deletions",
    "Q": "code_documentation_time",
    "R": "jira_id",
    "S": "commit_id",
    "T": "sprint",
    "U": "comment"
}

tasks_to_json = []

i = schema['data_offset_row']
while sheet[schema['columns']['WBS'] + str(i)].value is not None:
    if len(str(sheet[schema['columns']['Name'] + str(i)].value)) != 0:
        task = {}
        for column in map:
            task[map[column]] = str(sheet[column + str(i)].value)
        tasks_to_json.append(task)
    i = i + 1 

print(tasks_to_json)

[{'wbs': '0', 'name': 'Project configuration', 'description': 'Create a project from a template and add library requirements', 'dependency': 'None', 'estimated_development_time_designer': '1.5', 'estimated_development_time_reviewer': '1', 'estimated_development_time_pm': '1', 'assigned_developer': 'None', 'estimated_testing_time_tester': 'None', 'estimated_testing_time_pm': 'None', 'assigned_tester': 'None', 'actual_development_time_start': 'None', 'actual_development_time_finish': 'None', 'actual_development_time_duration': '1.5', 'commit_statistics_insertions': 'None', 'commit_statistics_deletions': 'None', 'code_documentation_time': 'None', 'jira_id': 'PP-458', 'commit_id': 'None', 'sprint': 'None', 'comment': 'None'}, {'wbs': '1', 'name': 'Describing abstract classes', 'description': 'Describe abstract classes and helper classes (like classes in model.py) ', 'dependency': '0', 'estimated_development_time_designer': '1', 'estimated_development_time_reviewer': 'None', 'estimated_deve

'213'

[]

In [18]:
from openpyxl import load_workbook
from openpyxl.styles import Font

workbook = load_workbook(filename='summary.xlsx')
sheet = workbook['Sheet1']

i = 1
while sheet['A' + str(i)].value is not None:

    print(sheet['A' + str(i)].value)
    print(sheet['c'+str(i)].value + sheet['d'+str(i)].value + sheet['e'+str(i)].value)
    issue = jira.issue_create({
            'summary': 'Assign categories to ' + sheet['d'+str(i)].value + ' testcases',
            # 'description': sheet[schema['columns']['Description'] + str(index)].value,
            'project': {'key': 'PP'},
            'issuetype': {'name': 'Sub-task'},
            'parent': {'key': 'PP-162'},
            # 'Priority': 'priority',
            # 'Status': 'status',
            # 'Creator': 'creator',
            # 'Assignee': 'assignee',
        })

    i = i + 1

Creating issue "Assign categories to View Dashboard testcases"
PP-160
Link View Dashboard testcases
Creating issue "Assign categories to Graph View testcases"
PP-160
Link Graph View testcases
Creating issue "Assign categories to Map View testcases"
PP-160
Link Map View testcases
Creating issue "Assign categories to 2D Graph View testcases"
PP-160
Link 2D Graph View testcases
Creating issue "Assign categories to Authentication testcases"
PP-160
Link Authentication testcases
Creating issue "Assign categories to General Sidebar Interactions testcases"
PP-160
Link General Sidebar Interactions testcases
Creating issue "Assign categories to Dataset Management testcases"
PP-160
Link Dataset Management testcases
Creating issue "Assign categories to Location Filter testcases"
PP-160
Link Location Filter testcases
Creating issue "Assign categories to Service Line Filter testcases"
PP-160
Link Service Line Filter testcases
Creating issue "Assign categories to Entity Filter testcases"
PP-160
Link 

In [26]:
print('February 22, 2021, 5:13 PM')

February 22, 2021, 5:13 PM


In [9]:
from dateutil import parser

parser.parse(None)

TypeError: Parser must be a string or character stream, not NoneType

In [70]:
import datetime
parser.parse('2021-5-5') + datetime.timedelta(hours = 5.5)


datetime.datetime(2021, 5, 5, 5, 30)